In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('Book_SQL').getOrCreate()

In [3]:
book_df = spark.read.csv('/content/Books.csv', sep=';', header=True, inferSchema=True)
user_df = spark.read.csv('/content/Users.csv', sep=';', header=True, inferSchema=True)
rate_df = spark.read.csv('/content/Book-Ratings.csv', sep=';', header=True, inferSchema=True)

In [4]:
book_df.printSchema()

root
 |-- ISBN: string (nullable = true)
 |-- BookTitle: string (nullable = true)
 |-- BookAuthor: string (nullable = true)
 |-- YearOfPublication: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- ImageURLS: string (nullable = true)
 |-- ImageURLM: string (nullable = true)
 |-- ImageURLL: string (nullable = true)



In [7]:
book_df_filtered=book_df.select('isbn', 'booktitle')
book_df_filtered.show(5)

+----------+--------------------+
|      isbn|           booktitle|
+----------+--------------------+
|0195153448| Classical Mythology|
|0002005018|        Clara Callan|
|0060973129|Decision in Normandy|
|0374157065|Flu: The Story of...|
|0393045218|The Mummies of Ur...|
+----------+--------------------+
only showing top 5 rows



In [8]:
user_df.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- USERNAME: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)



In [9]:
user_df_filtered=user_df.select('UserId', 'UserName')
user_df_filtered.show(5)

+------+-----------+
|UserId|   UserName|
+------+-----------+
|     1|bzsufoRTLN2|
|     2|fq7kfHg4VEI|
|     3|W0Hbkd3xR8v|
|     4|W51GahAx5Ap|
|     5|VKN3PQ18GgN|
+------+-----------+
only showing top 5 rows



In [10]:
rate_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- rate: integer (nullable = true)



In [11]:
rate_df.show(5)

+------+----------+----+
|userid|      isbn|rate|
+------+----------+----+
|276725|034545104X|   0|
|276726|0155061224|   5|
|276727|0446520802|   0|
|276729|052165615X|   3|
|276729|0521795028|   6|
+------+----------+----+
only showing top 5 rows



In [12]:
book_df_filtered.createOrReplaceTempView('Book')
user_df_filtered.createOrReplaceTempView('User')
rate_df.createOrReplaceTempView('Rate')

In [34]:
Query="""SELECT U.UserName,B.BookTitle,R.Rate,R2.AvgRate
         FROM Rate R
         INNER JOIN Book B ON B.ISBN=R.ISBN
         INNER JOIN User U ON U.UserId=R.UserId
         INNER JOIN (SELECT ISBN,AVG(rate) AvgRate
                     FROM Rate
                     GROUP BY ISBN)R2 ON R2.ISBN=R.ISBN
      """
Result=spark.sql(Query)
Result.show(10,truncate=False)

+-----------+--------------------------------------------------------------+----+-------+
|UserName   |BookTitle                                                     |Rate|AvgRate|
+-----------+--------------------------------------------------------------+----+-------+
|6chdqlR3DC7|The Way Things Work: An Illustrated Encyclopedia of Technology|8   |8.0    |
|px70uymJ7k6|Mog's Christmas                                               |0   |0.0    |
|mjteD2ip2Lj|Mog's Christmas                                               |0   |0.0    |
|cHwJip4Kj4k|Liar                                                          |9   |9.0    |
|6VUiynjA3tV|The Prime of Miss Jean Brodie                                 |0   |0.0    |
|cHwJip4Kj4k|The Fighting Man                                              |9   |9.0    |
|a0EEWhgtsW8|First Among Equals                                            |0   |0.0    |
|Tupz6KKVgIq|Matter Of Honour                                              |0   |0.0    |
|OMqCFWvTB

In [35]:
Result.write.csv('/content/bookRate_output')